# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [51]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [52]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [53]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [54]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [55]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [56]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [57]:
gamma = 1
sigma = .005

In [58]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [59]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [60]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.


Pregunta 2:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .6574


Pregunta 3:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 4:

Considere un swap a 1Y con cupones semestrales de 1.3% (lineal). Si la tasa
cupón cero a 0.5Y es 1.2% (exp), calcule la tasa cero a 1Y.


Pregunta 5:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,067.00 (365 días después).


Pregunta 6:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 482 y 654 días.


Pregunta 7:

Usando la curva cero cupón entregada y paráme

## Respuestas

### Pregunta 1

En cuanto a la estructura de las formulas, ambos modelos se parecen bastante, ya que HW es una extensión del modelo
de Vasicek. Ambos modelos tienen reversión a la media, formulas analiticas para el calculo de los bonos, eso conlleva a 
ambos modelos puedan utilizar el calculo de una call/put (opciones) desde las formulas de Black-Scholes para el calculo
La diferencia radica en que para HW podemos implementar la medida Forward, afectando así al "theta", siguiendo con las 
simulaciones y la valuación final en una simulación por ejemplo. Además se dice que HW calibra perfectamente la curva
cupon cero de mercado

### Pregunta 2

### Pregunta 3

De manera superficial, primero podemos ver las diferencias en las ecuaciones que componen cada metodo:
i) Medida libre de riesgo:

g(r(t), t) = E(t)[ exp( -{Integral[t,T] ( r(s) )ds} ) * g(r(T), T)]       

* con [t,T] son los limites de la integral

ii) Medida Forward:

g(r(t), t) = Z(t, T) * E(t)^f[g(r(T), T)] 

*E(t)^f es la aplicacion de la esperanza en medida forward

**Podemos ver que la diferencia radica en el uso del bono cupón cero Z(t,T) que vence en T como numerario para el calculo del payoff.**
pd: si queremos explicitamente el valor de g(r(T),T) basta con despejarlo

### Pregunta 4

### Pregunta 5

In [61]:
ICP0 = 10000
ICP365 = 10067 
dd=365 #dias 
TNA=(ICP365/ICP0 - 1)*(360/365) #asumimos literal ese numero de días desde el enunciado
print(f'El valor de la TNA es = {TNA:.4%}')

El valor de la TNA es = 0.6608%


### Pregunta 6

In [62]:
#Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
#forward entre 482 y 654 días.
t1=482
t2=654
r1=curva(t1)
r2=curva(t2)
df1=math.exp(-r1 * t1 / 365)
df2=math.exp(-r2 * t2 / 365)
fwd12=(df1 / df2 - 1) * 360 / (t2 - t1)
print(f'La tasa forward entre el día 482 y 654 es : {fwd12:.8%}')

La tasa forward entre el día 482 y 654 es : 0.04105425%


### Pregunta 7

In [63]:
#Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
#valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.0%.
#Entregue el resultado 8 decimales y suponiendo un nocional de 1.

Opcion_Call=hw.zcb_call_put(hw.CallPut.CALL, .99, curva(0), 1, 2, math.exp(-curva(365)), math.exp(-curva(730/365)), gamma, sigma)

print(f'El valor de la Call es : {Opcion_Call}')

El valor de la Call es : 0.009879433012166206
